In [ ]:
import pickle
import pandas as pd
from tqdm import tqdm

In [ ]:
pd.set_option('display.max_rows',500)

In [ ]:
aui_info = []

with open('/data/Bodenreider_UMLS_DL/UMLS_VERSIONS/2020AB-ACTIVE/META/MRCONSO.RRF','r') as fp:
    
    for line in fp.readlines():
        line = line.split('|')
        cui = line[0]
        aui = line[7]
        string = line[-5]
        
        aui_info.append({'AUI':aui, 'CUI':cui, 'STR':string})
        
cui2sg = {}

with open('/data/Bodenreider_UMLS_DL/UMLS_VERSIONS/2020AB-ACTIVE/META/MRSTY.RRF','r') as fp:
    
    for line in fp.readlines():
        line = line.split('|')
        cui = line[0]
        sg = line[3]
        cui2sg[cui] = sg

In [ ]:
cui2aui = {}
aui2cui = {}
aui2sg = {}

cui_sg = []
cui_aui = []

for tup in aui_info:
    aui = tup['AUI']
    cui = tup['CUI']
    sg = cui2sg[cui]
    
    auis = cui2aui.get(cui, [])
    auis.append(aui)
    cui2aui[cui] = auis
    
    aui2cui[aui] = cui
    
    cui_sg.append((cui, sg))
    cui_aui.append((cui, aui))

In [ ]:
cui_df = pd.DataFrame(cui_sg, columns=['cui','sg']).drop_duplicates()
cui_aui_df = pd.DataFrame(cui_aui, columns=['cui','aui']).drop_duplicates()

In [ ]:
cui_sg_count = cui_df.groupby('cui').count()
cui_sg_count['num'] = 1
cui_sg_count.groupby('sg').count()

In [ ]:
cui_aui_count = cui_aui_df.groupby('cui').count()
cui_aui_count['num'] = 1
cui_aui_count.groupby('aui').count().sort_values('num',ascending=False)

In [ ]:
cui_df.groupby('sg').count().sort_values('cui',ascending=False)

In [ ]:
test_preds = pd.read_csv('errors_pred.csv')

In [ ]:
test_preds

In [ ]:
test_set = pd.read_csv('all_testset.csv')

In [ ]:
fps = pd.read_csv('/data/Bodenreider_UMLS_DL/thilini/EXPERIMENTS/error_analysis/FP_with_auis.csv',sep='|')

In [ ]:
fps

In [ ]:
def same_sg_edge(aui1, aui2):
    
    cui1 = aui2cui[aui1]
    cui2 = aui2cui[aui2]

    sg1 = cui2sg[cui1]
    sg2 = cui2sg[cui2]
    
    if sg1 == sg2:
        return True
    else:
        return False

inter_sg_fps = []
intra_sg_fps = []

present = 0
total_edges = 0

for i, row in tqdm(fps.iterrows()):    
    aui1 = row['aui1']
    aui2 = row['aui2']
    
    total_edges += 1
    
    if aui1 in aui2cui and aui2 in aui2cui:
        
        if same_sg_edge(aui1, aui2):
            intra_sg_fps.append(row)
        else:
            inter_sg_fps.append(row)
            
        present += 1

In [ ]:
len(inter_sg_fps)

In [ ]:
len(intra_sg_fps)

In [ ]:
aui2cui['A24114892']